# IOCTL

> 什么是ioctl，有什么用？

`ioctl`是一个在C语言中用于设备驱动编程的函数。它的全名是"input/output control"，用于对设备进行直接的硬件级别的操作。这个函数通常用于那些不能用普通的文件I/O操作来完成的任务。

一个字符设备驱动通常会实现设备打开、关闭、读、写等功能，在一些需要细分的情境下，如果需要扩展新的功能，通常以增设 `ioctl()` 命令的方式实现

它允许`用户空间的应用程序`向`驱动程序`发送控制和配置命令

总的来说，`ioctl` 的主要作用是对设备的一些特性进行控制，例如串口的传输波特率、马达的转速等等。它提供了一种与设备驱动程序进行通信的方式。

## 应用层实现

ioctl 函数的原型是：

In [ ]:
int ioctl(int fd, unsigned long request, ...);



参数解释：

- `fd`：文件描述符，通常是通过`open`函数打开设备文件后得到的。
- `request`：设备特定的请求代码，这个代码决定了你想要设备做什么。
- `...`：这是可选的参数，取决于`request`。

返回值：如果成功则返回0，失败则返回-1。

注意：`ioctl`函数是非标准的，因此在不同的系统和设备上可能会有不同的行为。

## 驱动层

这个函数通常在设备驱动程序的file_operations结构中定义

ioctl操作函数的原型如下：

In [ ]:
long (*ioctl) (struct file *, unsigned int, unsigned long);



这个函数有三个参数：

1. `struct file *`：这是一个指向文件结构的指针，它代表了用户空间应用程序打开的设备文件。

2. `unsigned int`：这是ioctl命令号，它指定了应用程序希望执行的操作。

3. `unsigned long`：这是一个可选的参数，它可以用来传递额外的数据。
   
返回值：成功为0，失败-1

在ioctl函数内部，驱动程序通常会使用一个switch语句来处理不同的ioctl命令。

每个命令可能会执行不同的操作，例如读取设备状态、配置设备参数、启动或停止设备等。

> ioctl 命令号

在 Linux 中，提供了一种 `ioctl` 命令的统一格式，将 32 位 `int` 型数据划分为四个位段。这样可以确保命令和设备是一一对应的，避免将正确的命令发给错误的设备，或者是把错误的命令发给正确的设备，或者是把错误的命令发给错误的设备。

![alt text](ioctl2.png)

1. `dir（direction)`，ioctl 命令访问模式（属性数据传输方向），占据 2 bit，可以为 _IOC_NONE、_IOC_READ、_IOC_WRITE、_IOC_READ | _IOC_WRITE，分别指示了四种访问模式：无数据、读数据、写数据、读写数据；

   | 命令参数 | 访问模式 |
   | ------ | ------ | 
   | _IOC_NONE | 无数据 |
   | _IOC_READ | 读数据 |
   | _IOC_WRITE | 写数据 |
   | _IOC_READ \| _IOC_WRITE | 读写数据 |

2. `type（device type）`，设备类型，占据 8 bit，在一些文献中翻译为 “幻数” 或者 “魔数”，可以为任意 char 型字符，例如 
   ‘a’、’b’、’c’ 等等，其主要作用是使 ioctl 命令有唯一的设备标识；
3. `nr（number）`，命令编号/序数，占据 8 bit，可以为任意 unsigned char 型数据，取值范围 0~255，如果定义了多个 ioctl 命令，通常从 0 开始编号递增；
4. `size`，涉及到 ioctl 函数 第三个参数 arg ，占据 13bit 或者 14bit（体系相关，arm 架构一般为 14 位），指定了 arg 的数据类型及长度，如果在驱动的 ioctl 实现中不检查，通常可以忽略该参数；


> 定义命令操作的宏

内核中提供了一组宏定义来定义提取命令中的字段信息
 
_IOC 是定义命令所使用的最底层的宏,它的定义如下： 

In [ ]:
#define _IOC(dir,type,nr,size) (((dir)<<_IOC_DIRSHIFT)| \
                               ((type)<<_IOC_TYPESHIFT)| \
                               ((nr)<<_IOC_NRSHIFT)| \
                               ((size)<<_IOC_SIZESHIFT))

它将四个部分通过移位合并在一起

如果要定义一个命令，这个命令带参数，并且是将数据写入驱动，则最高两个比特位是01；写入的参数是一个struct option的结构数据，占12字节，则比特29到16的10进制值应该是12；如果定义幻数为字母s，命令码为2，最终应该使用`_IOC(1,'s',0,12)`来定义

这个命令还可以用更方便的宏来定义： `_IOW('s', 2, struct option)`

In [ ]:
// 定义无参数的 ioctl 命令
#define _IO(type,nr)   _IOC(_IOC_NONE,(type),(nr),0)

//定义带读参数的ioctl命令（copy_to_user） size为类型名
#define _IOR(type,nr,size)  _IOC(_IOC_READ,(type),(nr),(_IOC_TYPECHECK(size)))

//定义带写参数的 ioctl 命令（copy_from_user） size为类型名
#define _IOW(type,nr,size)  _IOC(_IOC_WRITE,(type),(nr),(_IOC_TYPECHECK(size)))

//定义带读写参数的 ioctl 命令 size为类型名
#define _IOWR(type,nr,size) _IOC(_IOC_READ|_IOC_WRITE,(type),(nr),(_IOC_TYPECHECK(size)))


另有解析命令的宏

In [ ]:

/* used to decode ioctl numbers */
#define _IOC_DIR(nr)        (((nr) >> _IOC_DIRSHIFT) & _IOC_DIRMASK)
#define _IOC_TYPE(nr)       (((nr) >> _IOC_TYPESHIFT) & _IOC_TYPEMASK)
#define _IOC_NR(nr)         (((nr) >> _IOC_NRSHIFT) & _IOC_NRMASK)
#define _IOC_SIZE(nr)       (((nr) >> _IOC_SIZESHIFT) & _IOC_SIZEMASK)

> ioctl 函数注册

在Linux字符驱动中，ioctl操作函数是通过在设备驱动的`file_operations`结构体中注册的。`file_operations`结构体定义了驱动程序可以执行的一组操作，包括打开设备、读取设备、写入设备等。

要注册ioctl操作函数，你需要在`file_operations`结构体中设置`ioctl`字段。以下是一个例子：



In [ ]:
static long device_ioctl(struct file *file, unsigned int cmd, unsigned long arg)
{
    // 在这里处理ioctl命令
}

static struct file_operations fops = {
    .owner = THIS_MODULE,
    .open = device_open,
    .release = device_release,
    .read = device_read,
    .write = device_write,
    .unlocked_ioctl = device_ioctl, // 注册ioctl操作函数
};



在这个例子中，`device_ioctl`函数是ioctl操作函数。它被注册到`file_operations`结构体的`unlocked_ioctl`字段。

注意，从Linux 2.6.11开始，推荐使用`unlocked_ioctl`而不是`ioctl`。`unlocked_ioctl`不会在调用ioctl操作函数时持有大内核锁，这可以提高系统的并发性能。

> 字符驱动如何处理ioctl命令？

在Linux字符驱动中，处理ioctl命令的不同操作通常是通过在ioctl操作函数中使用switch语句来实现的。每个case对应一个特定的ioctl命令。

以下是一个简单的例子：



In [ ]:
#include <linux/ioctl.h>

#define IOCTL_CMD1 _IOR('q', 1, int32_t *)
#define IOCTL_CMD2 _IOW('q', 2, int32_t *)

static long device_ioctl(struct file *file, unsigned int cmd, unsigned long arg)
{
    switch (cmd) 
    {
        case IOCTL_CMD1:
            // 处理IOCTL_CMD1命令
            break;
        case IOCTL_CMD2:
            // 处理IOCTL_CMD2命令
            break;
        default:
            // 未知的ioctl命令
            return -EINVAL;
    }
    return 0;
}



在这个例子中，`IOCTL_CMD1`和`IOCTL_CMD2`是两个自定义的ioctl命令。它们使用`_IOR`和`_IOW`宏定义，这两个宏用于生成唯一的ioctl命令号。`'q'`是这些命令的类型，`1`和`2`是命令的编号，`int32_t *`是命令的参数类型。

在`device_ioctl`函数中，使用switch语句根据ioctl命令的类型执行不同的操作。如果接收到未知的ioctl命令，函数返回`-EINVAL`表示错误。

# PROC

proc 文件系统是 Linux 内核的一部分,proc 文件系统中的文件通常被称为虚拟文件，它在内核空间和用户空间之间打开了一个通信窗口，用于在运行时访问系统内核的操作。这个文件系统是动态的，存储的是当前内核运行状态的一系列特殊文件。用户可以通过查看 /proc 下的对应文件来获取系统各种状态的实时数据，如内存使用情况、CPU占用率、进程信息等。

总的来说，proc 文件系统是一个非常强大的工具，它提供了一种简单而直接的方式来查看和修改内核的运行状态

创建 proc 文件或目录的过程主要包括以下步骤：

+ 创建目录或文件：使用 proc_mkdir 创建目录，使用 proc_create 创建文件。

+ 注册接口：使用 proc_create 函数注册接口。

+ 实现读取处理程序：读取处理程序接收4个参数：文件对象、用户空间缓冲区等。

+ 实现写处理程序：写入处理程序类似于读取处理程序，唯一的区别是用户缓冲区类型是 const char 指针。

每个 proc 文件或目录都由一个 proc_dir_entry 结构体来描述。这个结构体包含了一些重要的字段，如文件操作函数 proc_fops，文件或目录的父目录 parent，子目录 subdir 等。

当你在 /proc 目录下创建一个文件或目录时，会生成一个 proc_dir_entry 结构体来管理这个文件或目录。而当你打开这个文件或目录时，会生成一个 proc_inode 结构体来连接到 VFS

## code : proc 的实现

In [ ]:
//vser.h
#ifndef _VSER_H
#define _VSER_H

struct option {
	unsigned int datab;   // 数据位
	unsigned int parity;  // 奇偶校验位
	unsigned int stopb;   // 停止位
};

#define VS_MAGIC	's'

#define VS_SET_BAUD	_IOW(VS_MAGIC, 0, unsigned int)
#define VS_GET_BAUD	_IOW(VS_MAGIC, 1, unsigned int)
#define VS_SET_FFMT	_IOW(VS_MAGIC, 2, struct option)
#define VS_GET_FFMT	_IOW(VS_MAGIC, 3, struct option)

#endif

In [ ]:
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/fs.h>
#include <linux/cdev.h>
#include <linux/kfifo.h>

#include <linux/ioctl.h>
#include <linux/uaccess.h>
#include <linux/proc_fs.h>
#include <linux/seq_file.h>

#include "vser.h"

#define VSER_MAJOR	256
#define VSER_MINOR	0
#define VSER_DEV_CNT	1
#define VSER_DEV_NAME	"vser"

struct vser_dev {
	unsigned int baud;
	struct option opt;
	struct cdev cdev;
	struct proc_dir_entry *pdir;  // 目录项
	struct proc_dir_entry *pdat;  // 数据项
};

DEFINE_KFIFO(vsfifo, char, 32);  // 定义一个32字节的FIFO
static struct vser_dev vsdev;  

static int vser_open(struct inode *inode, struct file *filp)
{
	return 0;
}

static int vser_release(struct inode *inode, struct file *filp)
{
	return 0;
}

static ssize_t vser_read(struct file *filp, char __user *buf, size_t count, loff_t *pos)
{
	int ret;
	unsigned int copied = 0;

	ret = kfifo_to_user(&vsfifo, buf, count, &copied);  // 从FIFO中读取数据

	return ret == 0 ? copied : ret;
}

static ssize_t vser_write(struct file *filp, const char __user *buf, size_t count, loff_t *pos)
{
	int ret;
	unsigned int copied = 0;

	ret = kfifo_from_user(&vsfifo, buf, count, &copied);

	return ret == 0 ? copied : ret;
}

static long vser_ioctl(struct file *filp, unsigned int cmd, unsigned long arg)
{
	if (_IOC_TYPE(cmd) != VS_MAGIC)  // 判断命令是否合法
		return -ENOTTY;

	switch (cmd) {
	case VS_SET_BAUD:
		vsdev.baud = arg;
		break;
	case VS_GET_BAUD:
		arg = vsdev.baud;
		break;
	case VS_SET_FFMT:
		if (copy_from_user(&vsdev.opt, (struct option __user *)arg, sizeof(struct option)))
			return -EFAULT;
		break;
	case VS_GET_FFMT:
		if (copy_to_user((struct option __user *)arg, &vsdev.opt, sizeof(struct option)))
			return -EFAULT;
		break;
	default:
		return -ENOTTY;
	}

	return 0;
}

static struct file_operations vser_ops = {
	.owner = THIS_MODULE,
	.open = vser_open,
	.release = vser_release,
	.read = vser_read,
	.write = vser_write,
	.unlocked_ioctl = vser_ioctl,
};

static int dat_show(struct seq_file *m, void *v)
{
	struct vser_dev *dev = m->private;

	seq_printf(m, "baudrate: %d\n", dev->baud);  // 打印波特率
	return seq_printf(m, "frame format: %d%c%d\n", dev->opt.datab, \
			dev->opt.parity == 0 ? 'N' : dev->opt.parity == 1 ? 'O': 'E', \
			dev->opt.stopb);
			// 打印数据位、奇偶校验位和停止位
			// 判断parity  0: N  1: O  2: E
}

static int proc_open(struct inode *inode, struct file *file)
{
	return single_open(file, dat_show, PDE_DATA(inode));
	// single_open函数会调用dat_show函数，PDE_DATA(inode)返回proc_dir_entry的私有数据
}

static struct file_operations proc_ops = {
	.owner = THIS_MODULE,
	.open = proc_open,
	.release = single_release,
	.read = seq_read,
	.llseek = seq_lseek,
};

static int __init vser_init(void)
{
	int ret;
	dev_t dev;

	dev = MKDEV(VSER_MAJOR, VSER_MINOR);
	ret = register_chrdev_region(dev, VSER_DEV_CNT, VSER_DEV_NAME);
	if (ret)
		goto reg_err;

	cdev_init(&vsdev.cdev, &vser_ops);
	vsdev.cdev.owner = THIS_MODULE;
	vsdev.baud = 115200;
	vsdev.opt.datab = 8;
	vsdev.opt.parity = 0;
	vsdev.opt.stopb = 1;

	ret = cdev_add(&vsdev.cdev, dev, VSER_DEV_CNT);
	if (ret)
		goto add_err;

	vsdev.pdir = proc_mkdir("vser", NULL);  // 创建一个目录项
	if (!vsdev.pdir)
		goto dir_err;
	vsdev.pdat = proc_create_data("info", 0, vsdev.pdir, &proc_ops, &vsdev);
	// 创建一个数据项，与目录项关联
	if (!vsdev.pdat)
		goto dat_err;

	return 0;

dat_err:
	remove_proc_entry("vser", NULL);
dir_err:
	cdev_del(&vsdev.cdev);
add_err:
	unregister_chrdev_region(dev, VSER_DEV_CNT);
reg_err:
	return ret;
}

static void __exit vser_exit(void)
{
	
	dev_t dev;

	dev = MKDEV(VSER_MAJOR, VSER_MINOR);

	remove_proc_entry("info", vsdev.pdir);
	remove_proc_entry("vser", NULL);

	cdev_del(&vsdev.cdev);
	unregister_chrdev_region(dev, VSER_DEV_CNT);
}

module_init(vser_init);
module_exit(vser_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A simple character device driver");
MODULE_ALIAS("virtual-serial");

驱动实现后可用下面的命令来验证：

```kernel
    # make
    # make modules_install
    # depmod
    # modprobe vser
    # cat /proc/ver/info
    baudrate: 115200
    frame format: 8N1
```

# IO

> 四种典型IO模型

|     | 阻塞 | 非阻塞 |
| --- | --- | --- |
| 同步 | 阻塞IO | 非阻塞IO |
| 异步 | I/O 多路复用 | 异步I/O |

> 什么是同步和异步？

在IO中，同步（synchronous）和异步（asynchronous）是两个重要的概念：

- 同步IO：在同步IO中，一个进程（或线程）发起一个IO操作后，必须等待IO操作的完成，才能继续执行下一个操作。这就像是你在餐馆点餐，必须等待服务员把餐送到你面前，你才能开始吃饭。
- 
- 异步IO：在异步IO中，一个进程发起一个IO操作后，不需要等待IO操作的完成，就可以直接执行下一个操作。这就像是你在餐馆点餐后，可以先做其他事情，等餐做好了，服务员会通知你。

同步和异步的区别在于IO操作的第二个步骤是否阻塞。如果实际的IO读写阻塞请求进程，那么就是同步IO，因此阻塞IO、非阻塞IO、IO复用、信号驱动IO都是同步IO。如果不阻塞，而是操作系统帮你做完IO操作再将结果返回给你，那么就是异步IO

> 什么是阻塞和非阻塞

在IO操作中，阻塞（blocking）和非阻塞（non-blocking）是两个重要的概念：

- 阻塞IO：在阻塞IO中，一个进程（或线程）发起一个IO操作后，如果数据还没有准备好，进程就会被挂起（阻塞），直到数据准备好为止。这就像是你在电话中等待对方的回答，你无法做其他的事情。
- 非阻塞IO：在非阻塞IO中，一个进程发起一个IO操作后，如果数据还没有准备好，进程不会被挂起，而是立即返回，进程可以继续做其他的事情。
- 
简单来说，阻塞是指用户空间（调用线程）一直在等待，而不能干别的事情；非阻塞是指用户空间（调用线程）拿到内核返回的状态值就返回自己的空间，IO操作可以干就干，不可以干，就去干别的事情。这两种方式都有各自的优缺点，具体使用哪种方式取决于具体的应用场景和需求。

## 非阻塞IO

> 什么是阻塞和非阻塞的工作方式？

阻塞：不操作就进入休眠
非阻塞：不操作就返回错误

> 程序编程的注意事项

1. 用户程序中，要在打开文件时指定是否以阻塞方式打开
   
```C
fd = open(argv[1],O_RDWR | O_NONBLOCK);  //以非阻塞方式打开设备文件
```

2. 要在设备结构体中添加读等待队列和写等待队列头

```C
struct mychar_dev
{
	struct cdev mydev;

	char mydev_buf[BUF_LEN];
	int curlen;

	wait_queue_head_t rq;     //读队列
	wait_queue_head_t wq;     //写队列
};
```
wait_queue_head_t 是一个等待队列头的数据结构，用于实现进程的等待唤醒机制

一个等待队列头是一个等待队列的头部，等待队列是一个双向链表，链表中的每个节点都是一个等待队列项

等待队列项是一个等待队列的节点，它包含了一个指向等待队列头的指针，一个指向等待队列项的指针，一个指向等待队列项的下一个指针，一个指向等待队列项的上一个指针

>> linux内核对于休眠和唤醒的管理

3. 在实现读写函数时，如果数据不可用，要添加当前是否采用非阻塞的判断：
   ```C
   if(数据可用)
   {
        if(file结构指针->成员f_flags & O_NONBLOCK)
            return -EAGAIN;

   }
   ```

4. `init_waitqueue_head(&head)` 函数在 Linux 内核中用于初始化等待队列头,初始化等待队列头指针，使 task_list 字段的 next 和 prev 两者都指向头部指针本身

5. `wait_event_interruptible(wq,condition)` 是 Linux 内核中使用的一个宏，用于阻止等待队列中的当前任务，直到某个条件变为 true 

   这个函数先将当前进程的状态设置成TASK_INTERRUPTIBLE，然后调用schedule()，而schedule()会将位于TASK_INTERRUPTIBLE状态的当前进程从runqueue队列中删除。从runqueue队列中删除的结果是，当前这个进程将不再参与调度，除非通过其他函数将这个进程重新放入这个runqueue队列中，这就是wake_up()的作用了。

   由于这一段代码位于一个由condition控制的for(;;)循环中，所以当由shedule()返回时(当然是被wake_up之后，通过其他进程的schedule()而再次调度本进程)，如果条件condition不满足，本进程将自动再次被设置为TASK_INTERRUPTIBLE状态，接下来执行schedule()的结果是再次被从runqueue队列中删除。这时候就需要再次通过wake_up重新添加到runqueue队列中。
  
   如此反复，直到condition为真的时候被wake_up.
  
   可见，成功地唤醒一个被wait_event_interruptible()的进程，需要满足：
  
      在 1)condition为真的前提下，2) 调用wake_up()。

   所以，如果你仅仅修改condition，那么只是满足其中一个条件，这个时候，被wait_event_interruptible()起来的进程尚未位于runqueue队列中，因此不会被 schedule。这个时候只要wake_up一下就立刻会重新进入运行调度。
  
   关于wait_event_interruptible的返回值
  
   根据 wait_event_interruptible 的宏定义知：
  
      1) 条件condition为真时调用这个函数将直接返回0，而当前进程不会
      被 wait_event_interruptible和从runqueue队列中删除。
  
      2) 如果要被wait_event_interruptible的当前进程有nonblocked pending
      signals, 那么会直接返回-ERESTARTSYS(i.e. -512)，当前进程不会
      被wait_event_interruptible 和从runqueue队列中删除。
  
      3) 其他情况下，当前进程会被正常的wait_event_interruptible，并从
      runqueue队列中删除，进入TASK_INTERRUPTIBLE状态退出运行调度，
      直到再次被唤醒加入runqueue队列中后而参与调度，将正常返回0。 

6. `wait_event(wq, condition)` 是 Linux 内核中的一个宏，用于阻塞当前任务，直到某个条件成立
      注意，wait_event() 是不可中断的睡眠，条件一直不满足，会一直睡眠。如果你需要可中断的睡眠，可以使用 wait_event_interruptible()。如果你需要在超过指定的 timeout 时间后唤醒进程，可以使用 wait_event_timeout()。

7. `wake_up_interruptible(wait_queue_head_t *pwq)` 是 Linux 内核中的一个函数，用于唤醒注册到等待队列上的进程 
        这个函数不能直接立即唤醒进程，而是由调度程序转换上下文，调整为可运行状态
8. `wake_up(wait_queue_head_t *pwq)`和上面的功能一致，只不过能唤醒的程序类型不同。
   

   + wake_up(wait_queue_head_t *pwq)：此函数唤醒等待队列 pwq 中的所有进程，无论它们的状态是 TASK_INTERRUPTIBLE 还是 TASK_UNINTERRUPTIBLE。也就是说，它可以唤醒所有在等待队列中的进程。

   + wake_up_interruptible(wait_queue_head_t *pwq)：此函数只唤醒等待队列 pwq 中状态为 TASK_INTERRUPTIBLE 的进程。也就是说，它只唤醒那些可以被信号中断的进程

## code:非阻塞IO的实现1

In [ ]:
#ifndef _VSER_H
#define _VSER_H

struct option {
	unsigned int datab;
	unsigned int parity;
	unsigned int stopb;
};

#define VS_MAGIC	's'

#define VS_SET_BAUD	_IOW(VS_MAGIC, 0, unsigned int)
#define VS_GET_BAUD	_IOW(VS_MAGIC, 1, unsigned int)
#define VS_SET_FFMT	_IOW(VS_MAGIC, 2, struct option)
#define VS_GET_FFMT	_IOW(VS_MAGIC, 3, struct option)

#endif

In [ ]:
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/fs.h>
#include <linux/cdev.h>
#include <linux/kfifo.h>

#include <linux/ioctl.h>
#include <linux/uaccess.h>

#include "vser.h"

#define VSER_MAJOR	256
#define VSER_MINOR	0
#define VSER_DEV_CNT	1
#define VSER_DEV_NAME	"vser"

struct vser_dev {
	unsigned int baud;
	struct option opt;
	struct cdev cdev;
};

DEFINE_KFIFO(vsfifo, char, 32);
static struct vser_dev vsdev;

static int vser_open(struct inode *inode, struct file *filp)
{
	return 0;
}

static int vser_release(struct inode *inode, struct file *filp)
{
	return 0;
}

static ssize_t vser_read(struct file *filp, char __user *buf, size_t count, loff_t *pos)
{
	int ret;
	unsigned int copied = 0;

	if (kfifo_is_empty(&vsfifo))
// 数据不可用
		if (filp->f_flags & O_NONBLOCK)
// 非阻塞
			return -EAGAIN;
// 返回错误

	ret = kfifo_to_user(&vsfifo, buf, count, &copied);

	return ret == 0 ? copied : ret;
}

static ssize_t vser_write(struct file *filp, const char __user *buf, size_t count, loff_t *pos)
{
	int ret;
	unsigned int copied = 0;

	if (kfifo_is_full(&vsfifo))
		if (filp->f_flags & O_NONBLOCK)
			return -EAGAIN;

	ret = kfifo_from_user(&vsfifo, buf, count, &copied);

	return ret == 0 ? copied : ret;
}

static long vser_ioctl(struct file *filp, unsigned int cmd, unsigned long arg)
{
	if (_IOC_TYPE(cmd) != VS_MAGIC)
		return -ENOTTY;

	switch (cmd) {
	case VS_SET_BAUD:
		vsdev.baud = arg;
		break;
	case VS_GET_BAUD:
		arg = vsdev.baud;
		break;
	case VS_SET_FFMT:
		if (copy_from_user(&vsdev.opt, (struct option __user *)arg, sizeof(struct option)))
			return -EFAULT;
		break;
	case VS_GET_FFMT:
		if (copy_to_user((struct option __user *)arg, &vsdev.opt, sizeof(struct option)))
			return -EFAULT;
		break;
	default:
		return -ENOTTY;
	}

	return 0;
}

static struct file_operations vser_ops = {
	.owner = THIS_MODULE,
	.open = vser_open,
	.release = vser_release,
	.read = vser_read,
	.write = vser_write,
	.unlocked_ioctl = vser_ioctl,
};

static int __init vser_init(void)
{
	int ret;
	dev_t dev;

	dev = MKDEV(VSER_MAJOR, VSER_MINOR);
	ret = register_chrdev_region(dev, VSER_DEV_CNT, VSER_DEV_NAME);
	if (ret)
		goto reg_err;

	cdev_init(&vsdev.cdev, &vser_ops);
	vsdev.cdev.owner = THIS_MODULE;
	vsdev.baud = 115200;
	vsdev.opt.datab = 8;
	vsdev.opt.parity = 0;
	vsdev.opt.stopb = 1;

	ret = cdev_add(&vsdev.cdev, dev, VSER_DEV_CNT);
	if (ret)
		goto add_err;

	return 0;

add_err:
	unregister_chrdev_region(dev, VSER_DEV_CNT);
reg_err:
	return ret;
}

static void __exit vser_exit(void)
{
	
	dev_t dev;

	dev = MKDEV(VSER_MAJOR, VSER_MINOR);

	cdev_del(&vsdev.cdev);
	unregister_chrdev_region(dev, VSER_DEV_CNT);
}

module_init(vser_init);
module_exit(vser_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A simple character device driver");
MODULE_ALIAS("virtual-serial");

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <sys/ioctl.h>
#include <fcntl.h>
#include <errno.h>

#include "vser.h"

int main(int argc, char *argv[])
{
	int fd;
	int ret;
	unsigned int baud;
	struct option opt = {8,1,1};
	char rbuf[32] = {0};
	char wbuf[32] = "xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx";

	fd = open("/dev/vser0", O_RDWR | O_NONBLOCK);
	if (fd == -1)
		goto fail;

	ret = read(fd, rbuf, sizeof(rbuf));
	if (ret < 0)
		perror("read");

	ret = write(fd, wbuf, sizeof(wbuf));
	if (ret < 0)
		perror("first write");

	ret = write(fd, wbuf, sizeof(wbuf));
	if (ret < 0)
		perror("second write");


	close(fd);
	exit(EXIT_SUCCESS);

fail:
	perror("noneblock test");
	exit(EXIT_FAILURE);
}


## 阻塞IO

In [ ]:
#include <sys/types.h>
#include <sys/stat.h>
#include <fcntl.h>
#include <unistd.h>
#include <sys/ioctl.h>

#include <stdio.h>

#include "mychar.h"
int main(int argc,char *argv[])
{
	int fd = -1;
	char buf[8] = "";
	int ret = 0;

	if(argc < 2)
	{
		printf("The argument is too few\n");
		return 1;
	}

	fd = open(argv[1],O_RDWR); 
	if(fd < 0)
	{
		printf("open %s failed\n",argv[1]);
		return 2;
	}

	ret = read(fd,buf,8);
	if(ret < 0)
	{
		printf("read data failed\n");
	}
	else
	{
		printf("buf=%s\n",buf);
	}

	close(fd);
	fd = -1;
	return 0;
}


In [ ]:
#ifndef MY_CHAR_H
#define MY_CHAR_H

#include <asm/ioctl.h>

#define MY_CHAR_MAGIC 'k'

#define MYCHAR_IOCTL_GET_MAXLEN _IOR(MY_CHAR_MAGIC,1,int*)
#define MYCHAR_IOCTL_GET_CURLEN _IOR(MY_CHAR_MAGIC,2,int*)


#endif

In [ ]:
#include <linux/module.h>
#include <linux/kernel.h>
#include <linux/fs.h>
#include <linux/cdev.h>
#include <asm/uaccess.h>
#include <linux/wait.h>
#include <linux/sched.h>


#include "mychar.h"

#define BUF_LEN 100

#define MYCHAR_DEV_CNT 3

int major = 11;
int minor = 0;
int mychar_num  = MYCHAR_DEV_CNT;

struct mychar_dev
{
	struct cdev mydev;

	char mydev_buf[BUF_LEN];
	int curlen;

	wait_queue_head_t rq;     //读队列
	wait_queue_head_t wq;     //写队列
};

struct mychar_dev gmydev;

int mychar_open(struct inode *pnode,struct file *pfile)
{
	pfile->private_data =(void *) (container_of(pnode->i_cdev,struct mychar_dev,mydev));
	printk("mychar_open is called\n");
	return 0;
}

int mychar_close(struct inode *pnode,struct file *pfile)
{

	printk("mychar_close is called\n");
	return 0;
}

ssize_t mychar_read(struct file *pfile,char __user *puser,size_t count,loff_t *p_pos)
{
	struct mychar_dev *pmydev = (struct mychar_dev *)pfile->private_data;
	int size = 0;
	int ret = 0;

	if(pmydev->curlen <= 0)  // 如果缓冲区中没有数据
	{
		// struct file 结构体中的f_flags成员是一个标志位，用于存储文件描述符的标志
		//用户程序通过open函数打开文件时，可以指定成以非阻塞方式打开文件，此时f_flags中的O_NONBLOCK标志位会被置位
		if(pfile->f_flags & O_NONBLOCK)  // O_NONBLOCK是一个宏，用于判断文件描述符的标志是否为非阻塞
		{//非阻塞
			printk("O_NONBLOCK No Data Read\n");
			return -1;
		}
		else
		{//阻塞
			ret = wait_event_interruptible(pmydev->rq,pmydev->curlen > 0);  
			// wait_event_interruptible函数用于将进程加入到等待队列中，等待队列头是pmydev->rq，等待队列项是pmydev->curlen > 0
			// 如果pmydev->curlen > 0，说明缓冲区中有数据，此时不会阻塞
			if(ret)
			{
				printk("Wake up by signal\n");
				return -ERESTARTSYS;  // ERESTARTSYS是一个宏，用于返回系统重启
			}
		}
	}

	if(count > pmydev->curlen)
	{
		size = pmydev->curlen;
	}
	else
	{
		size = count;
	}

	ret = copy_to_user(puser, pmydev->mydev_buf, size);
	if(ret)
	{
		printk("copy_to_user failed\n");
		return -1;
	}
	memcpy(pmydev->mydev_buf, pmydev->mydev_buf + size, pmydev->curlen - size); // 将缓冲区中的数据前移
	//第一个参数是目的地址，第二个参数是源地址，第三个参数是拷贝的字节数
	// 拷贝的字节数是pmydev->curlen - size，即缓冲区中的数据长度减去读取的数据长度
	pmydev->curlen -= size;  // 更新缓冲区中的数据长度
    
	// 有数据读取，唤醒写队列
	wake_up_interruptible(&pmydev->wq);
		
	return size;
}

ssize_t mychar_write(struct file *pfile,const char __user *puser,size_t count,loff_t *p_pos)
{
	int size = 0;
	int ret = 0;
	struct mychar_dev *pmydev = (struct mychar_dev *)pfile->private_data;

	if(pmydev->curlen >= BUF_LEN)  // 如果字符缓冲区中的数据长度大于等于缓冲区的长度,说明缓冲区已满，此时应该阻塞
	{
		if(pfile->f_flags & O_NONBLOCK)  
		{
			printk("O_NONBLOCK Can not write data\n");
			return -1;
		}
		else
		{
			ret = wait_event_interruptible(pmydev->wq, pmydev->curlen < BUF_LEN);
			if(ret)
			{
				printk("wake up by signal\n");
				return -ERESTARTSYS;
			}
		}
	}


	if(count > BUF_LEN-pmydev->curlen)
	{
		size = BUF_LEN - pmydev->curlen;
	}
	else
	{
		size = count;
	}

	ret = copy_from_user(pmydev->mydev_buf + pmydev->curlen,puser,size);
	if(ret)
	{
		printk("copy_from_user failed\n");
		return -1;
	}
	pmydev->curlen  +=  size;  // 更新缓冲区中的数据长度

    // 有数据写入，唤醒读队列
	wake_up_interruptible(&pmydev->rq);

	return size;
}

long mychar_ioctl(struct file *pfile, unsigned int cmd, unsigned long arg)
{
	int __user *pret = (int *)arg;  //__user是一个宏，用于标识用户空间的指针
	int maxlen = BUF_LEN;
	int ret = 0;
	struct mychar_dev *pmydev = (struct mychar_dev *)pfile->private_data;

	switch(cmd)
	{
		case MYCHAR_IOCTL_GET_MAXLEN:
			ret = copy_to_user(pret, &maxlen,sizeof(int));
			if(ret)
			{
				printk("copy_to_user MAXLEN failed\n");
				return -1;
			}
			break;
		case MYCHAR_IOCTL_GET_CURLEN:
			ret = copy_to_user(pret, &pmydev->curlen,sizeof(int));
			if(ret)
			{
				printk("copy_to_user CURLEN failed\n");
				return -1;
			}
			break;
		default:
			printk("The cmd is unknow\n");
			return -1;

	}

	return 0;
}

struct file_operations myops = {
	.owner = THIS_MODULE,
	.open = mychar_open,
	.release = mychar_close,
	.read = mychar_read,
	.write = mychar_write,
	.unlocked_ioctl = mychar_ioctl,
};

int __init mychar_init(void)
{
	int ret = 0;
	dev_t devno = MKDEV(major,minor);
	int i = 0;

	/*申请设备号*/
	ret = register_chrdev_region(devno,mychar_num,"mychar");
	if(ret)
	{
		ret = alloc_chrdev_region(&devno,minor,mychar_num,"mychar");
		if(ret)
		{
			printk("get devno failed\n");
			return -1;
		}
		major = MAJOR(devno);//容易遗漏，注意
	}


		/*给struct cdev对象指定操作函数集*/	
		cdev_init(&gmydev.mydev,&myops);

		/*将struct cdev对象添加到内核对应的数据结构里*/
		gmydev.mydev.owner = THIS_MODULE;
		cdev_add(&gmydev.mydev,devno,1);

	init_waitqueue_head(&gmydev->rq);  // 初始化读队列 函数原型 void init_waitqueue_head(wait_queue_head_t *q)

	init_waitqueue_head(&gmydev->wq);

	return 0;
}

void __exit mychar_exit(void)
{
	dev_t devno = MKDEV(major,minor);
	int i = 0;

	cdev_del(&gmydev_arr[i].mydev);
	

	unregister_chrdev_region(devno,mychar_num);
}


MODULE_LICENSE("GPL");

module_init(mychar_init);
module_exit(mychar_exit);

# 异步IO

使用异步IO
> 在应用层

1. 需要定义 aiocb 异步控制块 struct aiocb aiow aior;
2. 需要初始化控制块
    + 清空结构体信息 memset
    + 给结构体的成员配置新的值 
      + 文件描述符——io操作的对象
      + aio缓冲区——接收传递的数据
      + 通知的方式
      + 回调函数
3. 在应用层可以调用aio_write和aio_read函数来发起异步读和异步写
4. 完成读写后会调用初始化时定义的sigev_notify_function函数
5. 最后读写完成的状态交由在回调函数中使用aio_return和aio_error来处理结果
   
> 在驱动层

1. 在file_operation中注册操作函数
2. 在操作函数中会调用读写函数xxx_read
3. 调用会以一种多次进行的方式，每次读取一定的字符数，然后分散地放入分散的内存区域中


`aiocb` 是 C 语言中的一个结构体，全名为 Asynchronous Input/Output Control Block。它用于异步 I/O 操作，允许程序在没有完成 I/O 操作的情况下继续执行其他任务。这个结构体通常包含以下字段：

- `int aio_fildes`：文件描述符
- `off_t aio_offset`：文件偏移量
- `volatile void *aio_buf`：数据缓冲区
- `size_t aio_nbytes`：传输的字节数
- `int aio_reqprio`：请求的优先级
- `struct sigevent aio_sigevent`：完成异步 I/O 后要发送的信号
- `int aio_lio_opcode`：操作类型

在使用 `aiocb` 结构体进行异步 I/O 操作时，通常需要先初始化这个结构体，然后使用 `aio_read` 或 `aio_write` 函数进行读写操作。

`aio_read` 和 `aio_write` 是 C 语言中的异步 I/O 函数，它们分别用于异步读取和写入数据。这两个函数的原型如下：



In [ ]:
#include <aio.h>

int aio_read(struct aiocb *aiocbp);
int aio_write(struct aiocb *aiocbp);

这两个函数的参数都是一个指向 `aiocb` 结构体的指针，这个结构体包含了进行异步 I/O 操作所需的所有信息，如文件描述符、偏移量、缓冲区地址和大小等。

这两个函数在调用后会立即返回，不会等待 I/O 操作完成。
你可以使用 `aio_error` 函数来检查 I/O 操作是否完成，使用 `aio_return` 函数来获取 I/O 操作的结果。这会写在aiow_completion_handler里

+ aio_error：aio_error函数返回异步I/O请求的错误状态1。这个函数的返回值有以下几种可能：
    + 如果异步I/O操作已经成功完成，aio_error返回0。
    + 如果异步I/O操作未成功完成，aio_error返回错误状态，这个错误状态与对应的read()或write()函数描述的一致。
    + 如果异步I/O操作还未完成，那么返回EINPROGRESS。

+ aio_return：aio_return函数返回异步I/O请求的最终返回状态。这个函数只应在aio_error返回除EINPROGRESS外的其他值后被调用一次。
  + 如果异步I/O操作已经完成，这个函数返回的值与对应的同步read()、write()、fsync()或fdatasync()调用的返回值一致。
  + 如果异步I/O操作还未完成，aio_return的返回值和效果是未定义的。


## code : 这是一个使用 `aio_read` 和 `aio_write` 的简单示例：

In [ ]:
#include <aio.h>
#include <fcntl.h>
#include <stdio.h>
#include <string.h>
#include <unistd.h>

int main() {
    struct aiocb cb;
    int fd, ret;

    fd = open("file.txt", O_RDONLY);
    if (fd == -1) {
        perror("open");
        return 1;
    }

    memset(&cb, 0, sizeof(struct aiocb));
    cb.aio_nbytes = 100;
    cb.aio_fildes = fd;
    cb.aio_offset = 0;
    cb.aio_buf = malloc(cb.aio_nbytes+1);
    if (cb.aio_buf == NULL) {
        perror("malloc");
        close(fd);
        return 1;
    }

    ret = aio_read(&cb);
    if (ret < 0) {
        perror("aio_read");
        close(fd);
        return 1;
    }

    while (aio_error(&cb) == EINPROGRESS) { // 等待异步操作完成
        printf("waiting...\n");
        sleep(1);
    }

    ret = aio_return(&cb);
    if (ret > 0) {
        ((char*)cb.aio_buf)[ret] = 0;
        printf("%s\n", cb.aio_buf);
    }

    free((void*)cb.aio_buf);
    close(fd);

    return 0;
}



这个程序会异步地从 `file.txt` 文件中读取 100 个字节的数据，然后打印出来。

## code: 虚拟串口驱动中实现异步IO

In [ ]:
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <sys/ioctl.h>
#include <fcntl.h>
#include <errno.h>
#include <poll.h>
#include <linux/input.h>
#include <aio.h>

#include "vser.h"

void aiow_completion_handler(sigval_t sigval)
{
	int ret;
	struct aiocb *req;

	req = (struct aiocb *)sigval.sival_ptr;

	if (aio_error(req) == 0) {
		ret = aio_return(req);
		printf("aio write %d bytes\n", ret);
	}

	return;
}

void aior_completion_handler(sigval_t sigval)
{
	int ret;
	struct aiocb *req;

	req = (struct aiocb *)sigval.sival_ptr;

	if (aio_error(req) == 0) {
		ret = aio_return(req);
		if (ret)
			printf("aio read: %s\n", (char *)req->aio_buf);
	}

	return;
}

int main(int argc, char *argv[])
{
	int ret;
	int fd;
	struct aiocb aiow, aior; //异步IO控制块，相当于定义两个操作读写的对象

	fd = open("/dev/vser0", O_RDWR);
	if (fd == -1) 
		goto fail;
	
	//初始化异步IO控制块
	//清空其中的内容
	memset(&aiow, 0, sizeof(aiow));  
	memset(&aior, 0, sizeof(aior));
	
	//给读异步IO控制块配置参数
	aiow.aio_fildes = fd;   // 传入文件描述符
	aiow.aio_buf = malloc(32);  // 传入申请32字节的空间的地址作为数据缓冲区
	strcpy((char *)aiow.aio_buf, "aio test");  //要把aio_buf的类型先转换为char*，然后再赋值
	aiow.aio_nbytes = strlen((char *)aiow.aio_buf) + 1; // 传入数据长度
	aiow.aio_offset = 0;  //偏移量
	aiow.aio_sigevent.sigev_notify = SIGEV_THREAD;  // 通知方式 SIGEV_THREAD表示使用线程
	aiow.aio_sigevent.sigev_notify_function = aiow_completion_handler;  // 通知的回调函数
	aiow.aio_sigevent.sigev_notify_attributes = NULL;  // 通知的属性
	aiow.aio_sigevent.sigev_value.sival_ptr = &aiow;  // 通知的参数sigval联合体的一个成员用于在发送信号时传递额外的数据
	// 给写异步IO控制块配置参数
	aior.aio_fildes = fd;  
	aior.aio_buf = malloc(32);
	aior.aio_nbytes = 32;
	aior.aio_offset = 0;
	aior.aio_sigevent.sigev_notify = SIGEV_THREAD;
	aior.aio_sigevent.sigev_notify_function = aior_completion_handler;
	aior.aio_sigevent.sigev_notify_attributes = NULL;
	aior.aio_sigevent.sigev_value.sival_ptr = &aior;

	while (1) 
	{
		if (aio_write(&aiow) == -1)
			goto fail;
		if (aio_read(&aior) == -1)
			goto fail;
		sleep(1);
	}

fail:
	perror("aio test");
	exit(EXIT_FAILURE);
}


In [ ]:
#ifndef _VSER_H
#define _VSER_H

struct option {
	unsigned int datab;
	unsigned int parity;
	unsigned int stopb;
};

#define VS_MAGIC	's'

#define VS_SET_BAUD	_IOW(VS_MAGIC, 0, unsigned int)
#define VS_GET_BAUD	_IOW(VS_MAGIC, 1, unsigned int)
#define VS_SET_FFMT	_IOW(VS_MAGIC, 2, struct option)
#define VS_GET_FFMT	_IOW(VS_MAGIC, 3, struct option)

#endif

In [ ]:
#include <linux/init.h>
#include <linux/kernel.h>
#include <linux/module.h>

#include <linux/fs.h>
#include <linux/cdev.h>
#include <linux/kfifo.h>

#include <linux/ioctl.h>
#include <linux/uaccess.h>

#include <linux/wait.h>
#include <linux/sched.h>
#include <linux/poll.h>
#include <linux/aio.h>

#include "vser.h"

#define VSER_MAJOR	256
#define VSER_MINOR	0
#define VSER_DEV_CNT	1
#define VSER_DEV_NAME	"vser"

struct vser_dev {
	unsigned int baud;
	struct option opt;
	struct cdev cdev;
	wait_queue_head_t rwqh;
	wait_queue_head_t wwqh;
};

DEFINE_KFIFO(vsfifo, char, 32);
static struct vser_dev vsdev;

static int vser_open(struct inode *inode, struct file *filp)
{
	return 0;
}

static int vser_release(struct inode *inode, struct file *filp)
{
	return 0;
}

static ssize_t vser_read(struct file *filp, char __user *buf, size_t count, loff_t *pos)
{
	int ret;
	unsigned int copied = 0;

	if (kfifo_is_empty(&vsfifo)) {
		if (filp->f_flags & O_NONBLOCK)
			return -EAGAIN;

		if (wait_event_interruptible_exclusive(vsdev.rwqh, !kfifo_is_empty(&vsfifo)))
			return -ERESTARTSYS;
	}

	ret = kfifo_to_user(&vsfifo, buf, count, &copied);

	if (!kfifo_is_full(&vsfifo))
		wake_up_interruptible(&vsdev.wwqh);

	return ret == 0 ? copied : ret;
}

static ssize_t vser_write(struct file *filp, const char __user *buf, size_t count, loff_t *pos)
{

	int ret;
	unsigned int copied = 0;

	if (kfifo_is_full(&vsfifo)) {
		if (filp->f_flags & O_NONBLOCK)
			return -EAGAIN;

		if (wait_event_interruptible_exclusive(vsdev.wwqh, !kfifo_is_full(&vsfifo)))
			return -ERESTARTSYS;
	}

	ret = kfifo_from_user(&vsfifo, buf, count, &copied);

	if (!kfifo_is_empty(&vsfifo))
		wake_up_interruptible(&vsdev.rwqh);

	return ret == 0 ? copied : ret;
}

static long vser_ioctl(struct file *filp, unsigned int cmd, unsigned long arg)
{
	if (_IOC_TYPE(cmd) != VS_MAGIC)
		return -ENOTTY;

	switch (cmd) {
	case VS_SET_BAUD:
		vsdev.baud = arg;
		break;
	case VS_GET_BAUD:
		arg = vsdev.baud;
		break;
	case VS_SET_FFMT:
		if (copy_from_user(&vsdev.opt, (struct option __user *)arg, sizeof(struct option)))
			return -EFAULT;
		break;
	case VS_GET_FFMT:
		if (copy_to_user((struct option __user *)arg, &vsdev.opt, sizeof(struct option)))
			return -EFAULT;
		break;
	default:
		return -ENOTTY;
	}

	return 0;
}

static unsigned int vser_poll(struct file *filp, struct poll_table_struct *p)
{
	int mask = 0;

	poll_wait(filp, &vsdev.rwqh, p);
	poll_wait(filp, &vsdev.wwqh, p);

	if (!kfifo_is_empty(&vsfifo))
		mask |= POLLIN | POLLRDNORM;
	if (!kfifo_is_full(&vsfifo))
		mask |= POLLOUT | POLLWRNORM;

	return mask;
}

static ssize_t vser_aio_read(struct kiocb *iocb, const struct iovec *iov, unsigned long nr_segs, loff_t pos)
{
	size_t read = 0;
	unsigned long i;
	ssize_t ret;

	for (i = 0; i < nr_segs; i++) {
		ret = vser_read(iocb->ki_filp, iov[i].iov_base, iov[i].iov_len, &pos);
		if (ret < 0)
			break;
		read += ret;
	}

	return read ? read : -EFAULT;
}

static ssize_t vser_aio_write(struct kiocb *iocb, const struct iovec *iov, unsigned long nr_segs, loff_t pos)
{
// kiocb是异步IO控制块，iov是用户空间的缓冲区，nr_segs是缓冲区的数量，pos是文件的偏移量
	size_t written = 0;
	unsigned long i;
	ssize_t ret;

	for (i = 0; i < nr_segs; i++) {//遍历缓冲区
		ret = vser_write(iocb->ki_filp, iov[i].iov_base, iov[i].iov_len, &pos);
		//调用vser_write函数，将ki_filp指向的文件写入到缓冲区iov[i].iov_base中
		if (ret < 0)
			break;
		written += ret;
	}

	return written ? written : -EFAULT;
}

static struct file_operations vser_ops = {
	.owner = THIS_MODULE,
	.open = vser_open,
	.release = vser_release,
	.read = vser_read,
	.write = vser_write,
	.unlocked_ioctl = vser_ioctl,
	.poll = vser_poll,
	.aio_read = vser_aio_read,  // 异步读
	.aio_write = vser_aio_write,  //异步写
};

static int __init vser_init(void)
{
	int ret;
	dev_t dev;

	dev = MKDEV(VSER_MAJOR, VSER_MINOR);
	ret = register_chrdev_region(dev, VSER_DEV_CNT, VSER_DEV_NAME);
	if (ret)
		goto reg_err;

	cdev_init(&vsdev.cdev, &vser_ops);
	vsdev.cdev.owner = THIS_MODULE;
	vsdev.baud = 115200;
	vsdev.opt.datab = 8;
	vsdev.opt.parity = 0;
	vsdev.opt.stopb = 1;

	ret = cdev_add(&vsdev.cdev, dev, VSER_DEV_CNT);
	if (ret)
		goto add_err;

	init_waitqueue_head(&vsdev.rwqh);  // 初始化等待队列头
	init_waitqueue_head(&vsdev.wwqh);  // 初始化等待队列头

	return 0;

add_err:
	unregister_chrdev_region(dev, VSER_DEV_CNT);
reg_err:
	return ret;
}

static void __exit vser_exit(void)
{
	
	dev_t dev;

	dev = MKDEV(VSER_MAJOR, VSER_MINOR);

	cdev_del(&vsdev.cdev);
	unregister_chrdev_region(dev, VSER_DEV_CNT);
}

module_init(vser_init);
module_exit(vser_exit);

MODULE_LICENSE("GPL");
MODULE_AUTHOR("Kevin Jiang <jiangxg@farsight.com.cn>");
MODULE_DESCRIPTION("A simple character device driver");
MODULE_ALIAS("virtual-serial");


如何处理多个异步io操作

# 异步通知